In [1]:
#load necessary packages
import os
from PIL import Image
import numpy as np
import shutil

In [2]:
import pandas as pd
file_list = pd.read_csv('../ULCER_SET_CLASSES', header = None)
print('Total ulcer Images:', file_list.shape[0])
print(file_list)

Total ulcer Images: 44893
                               0  1
0      DMS-RG-0840891-C-01_sppx1  0
1      DMS-RG-0840891-C-01_sppx2  0
2      DMS-RG-0840891-C-01_sppx3  0
3      DMS-RG-0840891-C-01_sppx4  0
4      DMS-RG-0840891-C-01_sppx5  0
...                          ... ..
44888           IMG_6537_sppx538  0
44889           IMG_6537_sppx539  0
44890           IMG_6537_sppx540  0
44891           IMG_6537_sppx541  0
44892           IMG_6537_sppx542  0

[44893 rows x 2 columns]


In [11]:
#record and print out the filename list that belong to group 0,1,2,3
file_list_group0 = file_list.loc[file_list[1] == 0, 0]
file_list_group1 = file_list.loc[file_list[1] == 1, 0]
file_list_group2 = file_list.loc[file_list[1] == 2, 0]
file_list_group3 = file_list.loc[file_list[1] == 3, 0]
print('not wound images : ', file_list_group0.shape[0])
print('fibrin images: ', file_list_group1.shape[0])
print('granulation images : ', file_list_group2.shape[0])
print('necrosed images : ', file_list_group3.shape[0])


not wound images :  37187
fibrin images:  3974
granulation images :  3284
necrosed images :  448


In [12]:
#example why need to add . to every item in the list
s = 'IMG_6537_sppx12.png'
print(s.startswith('IMG_6537_sppx1'))
print(s.startswith('IMG_6537_sppx1.'))
file_list_group0_d = [item + "." for item in file_list_group0]
file_list_group1_d = [item + "." for item in file_list_group1]
file_list_group2_d = [item + "." for item in file_list_group2]
file_list_group3_d = [item + "." for item in file_list_group3]
#print(file_list_group3_d)

True
False


In [13]:
#grouping and check if matching
source = "../ULCER_SET"
dest_g0 = "ulcer_dataset/g0"
dest_g1 = "ulcer_dataset/g1"
dest_g2 = "ulcer_dataset/g2"
dest_g3 = "ulcer_dataset/g3"

directories = os.listdir(source)
for f in directories:
    if (f.startswith(tuple(file_list_group0_d))):
        shutil.copy2(os.path.join(source,f), dest_g0)
    elif (f.startswith(tuple(file_list_group1_d))):
        shutil.copy2(os.path.join(source,f), dest_g1)
    elif (f.startswith(tuple(file_list_group2_d))):
        shutil.copy2(os.path.join(source,f), dest_g2)
    elif (f.startswith(tuple(file_list_group3_d))):
        shutil.copy2(os.path.join(source,f), dest_g3)

In [14]:
#Directory containing images you wish to convert
input_dir = "ulcer_dataset"
directories = os.listdir(input_dir)
print(directories)

['g0', 'g2', 'g3', '.ipynb_checkpoints', 'g1']


In [30]:
index = 0
index2 = 0

for folder in ['g0','g1','g2','g3']:
    #Ignoring .DS_Store dir
    print(folder)
    if folder == '.ipynb_checkpoints':
        continue
    else:
        print("The currently using folder is: ", folder)
        folder2 = os.listdir(input_dir + '/' + folder)
        index += 1
        for image in folder2:
            if image == ".ipynb_checkpoints":
                pass
            else:
                index2 += 1
                im = Image.open(input_dir+"/"+folder+"/"+image) #Opening image
                resized_im = im.resize((70,70), resample = Image.NEAREST ) # resized image of size 70x70, because origin images were just missing 1 or 2 pixels on edge using nearset neighbor method                
                im = (np.array(resized_im)) #Converting to numpy array
                
                try:
                    r = im[:,:,0] #Slicing to get R data
                    g = im[:,:,1] #Slicing to get G data
                    b = im[:,:,2] #Slicing to get B data
                
                    if index2 != 1:
                        new_array = np.array([[r] + [g] + [b]], np.uint8) #Creating array with shape (3, 70, 70)
                        out = np.append(out, new_array, 0) #Adding new image to array shape of (x, 3, 70, 70) where x is image number
                
                    elif index2 == 1:
                        out = np.array([[r] + [g] + [b]], np.uint8) #Creating array with shape (3, 70, 70)
                
                    if index == 1 and index2 == 1:
                        index_array = np.array([[index]])
                
                    else:
                        new_index_array = np.array([[index]], np.int8)
                        index_array = np.append(index_array, new_index_array, 0)
            
                except Exception as e:
                    print(e)
                    print("Removing image" + image)
                    #os.remove(input_dir+"/"+folder+"/"+image)

print("Number of folders/classes: ", index)

g0
The currently using folder is:  g0
g1
The currently using folder is:  g1
g2
The currently using folder is:  g2
g3
The currently using folder is:  g3
Number of folders/classes:  4


In [31]:
#saving the train image arrays and labels
np.save('features_train.npy', out) #Saving train image arrays
np.save('labels_train.npy', index_array) #Saving train labels